# <center> Google - Generativeai 🌐💻

In [1]:
# from dotenv import load_dotenv
# import os
# import google.generativeai as genai


# load_dotenv()
# api_key = os.getenv("GEMINI_API_KEY")
# genai.configure(api_key=api_key) 

In [2]:
# model = genai.GenerativeModel("gemini-2.0-flash")

# try:
#    response = model.generate_content(input("Digite sua pergunta: "))
#    print(response.text)
# except Exception as e:
#     print("Erro ao consumir a API:", e)


In [3]:
# import google.generativeai as genai
# from dotenv import load_dotenv
# import os

# load_dotenv()
# genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
# model = genai.GenerativeModel("gemini-2.0-flash")

# # Cria uma sessão de chat
# chat = model.start_chat()
# # Defina o prompt de sistema
# system_prompt = "Você é um assistente técnico em IA que responde de forma objetiva, clara e sem rodeios. Sempre utilize exemplos em Python."
# try:
#    # Envia o prompt de sistema como a primeira mensagem para definir o comportamento
#    chat.send_message(system_prompt)

#     # Agora o modelo pode receber a entrada do usuário
#    user_prompt = input("Digite sua pergunta: ")

#    response = chat.send_message(user_prompt)
#    print(response.text)
# except Exception as e:
#    print(f"Ocorreu um erro: {e}")

In [4]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain.schema.messages import SystemMessage, HumanMessage
# from dotenv import load_dotenv
# import os 

# load_dotenv()
# api_key = os.getenv("GEMINI_API_KEY")

# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash",
#     temperature=0.7,
#     google_api_key=api_key
# ) 

# messages = [
#     SystemMessage(
#         content="Você é um tutor especializado em LangChain. Responda sempre com clareza, de forma objetiva, com exemplos simples em Python. Evite jargões técnicos desnecessários."
#     ),
#     HumanMessage(
#         content="O que é LangChain e como ele se relaciona com o GEMINI?"
#     )
# ]

# try:
#     response = llm(messages)
#     print(response.content)
# except Exception as e:
#     print(f"Ocorreu um erro: {e}")


In [1]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate  # <-- IMPORT NECESSÁRIO

# Carrega as variáveis de ambiente
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

# Inicializa o modelo LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=api_key)

# Pasta com os arquivos
pasta_arquivos = "./arquivos/"

# Lista os arquivos da pasta
try:
    if os.path.isdir(pasta_arquivos):
        arquivos_disponiveis = [
            os.path.join(pasta_arquivos, nome)
            for nome in os.listdir(pasta_arquivos)
            if os.path.isfile(os.path.join(pasta_arquivos, nome))
        ]
    else:
        print(f"Pasta '{pasta_arquivos}' não encontrada.")
        arquivos_disponiveis = []
except Exception as e:
    print(f"Erro ao listar arquivos: {e}")
    arquivos_disponiveis = []

# Nomes dos arquivos
nomes_exibicao = [os.path.basename(arquivo) for arquivo in arquivos_disponiveis]
nomes_validos = set(nomes_exibicao)

# Função segura para leitura
def ler_arquivo(nome_arquivo: str) -> str:
    if nome_arquivo not in nomes_validos:
        return f"Erro: O arquivo '{nome_arquivo}' não está na lista de arquivos disponíveis."
    caminho_completo = os.path.join(pasta_arquivos, nome_arquivo)
    try:
        with open(caminho_completo, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"Erro ao ler o arquivo '{nome_arquivo}': {e}"

# Ferramenta
tools = [
    Tool(
        name="ler_arquivo",
        func=ler_arquivo,
        description=f"Lê o conteúdo de um arquivo de texto listado em: {', '.join(nomes_exibicao)}"
    )
]

# Memória e prompt
memory = ConversationBufferMemory(memory_key="chat_history")
mensagem_inicial = f"""Você é um especialista em gêneros musicais.
Use a ferramenta 'ler_arquivo' para obter informações sobre estilos musicais, origens e artistas.
NUNCA invente nomes de arquivos. Use somente os arquivos existentes:

Arquivos disponíveis: {', '.join(nomes_exibicao)}"""
memory.chat_memory.add_user_message(mensagem_inicial)

# Inicializa agente
agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
)

# Pergunta
pergunta_usuario = "Qual o maior cantor de musica pop?"
try:
    resposta_agente = agent.run(pergunta_usuario)
    print(f"\nResposta do Agente (baseada nos arquivos existentes): {resposta_agente}")
except Exception as e:
    print(f"Ocorreu um erro ao executar o agente: {e}")

# Juiz
try:
    llm_juiz = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17", google_api_key=api_key)

    prompt_juiz_template = """Você é um juiz de IA. Avalie se a seguinte afirmação é correta
(SIM ou NAO) e justifique: "{afirmacao}"."""
    prompt_juiz = ChatPromptTemplate.from_template(prompt_juiz_template)

    # Formata o prompt com a resposta do agente
    prompt_formatado = prompt_juiz.format_messages(afirmacao=resposta_agente)

    # Chama o modelo com o prompt já formatado
    output_juiz = llm_juiz(prompt_formatado)
    avaliacao_juiz = output_juiz.content

    print(f"\nAvaliação do Juiz (Gemini Pro):\n{avaliacao_juiz}")
    if "NAO" in avaliacao_juiz.upper():
        print("\n O Juiz identificou uma possível alucinação!")
    else:
        print("\n O Juiz considerou a informação factual.")
except Exception as e:
    print(f"Ocorreu um erro ao executar o juiz: {e}")


C:\Users\livianavarro-ieg\AppData\Local\Temp\ipykernel_15772\1871694576.py:59: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
C:\Users\livianavarro-ieg\AppData\Local\Temp\ipykernel_15772\1871694576.py:68: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = init



> Entering new AgentExecutor chain...
Para responder a essa pergunta, preciso analisar o conteúdo dos arquivos de texto disponíveis e identificar o cantor mais proeminente na categoria de música pop.

Action: ler_arquivo
Action Input: musica_pop.txt
Observation: Atualmente aborda temas românticos, especialmente sobre o amor entre jovens
Tornou-se um gênero musical popular em todo o mundo
América Latina possui artistas relevantes como Shakira, Jennifer Lopez e Ricky Martin
Michael Jackson é conhecido como "O Rei do Pop", considerado o maior cantor do gênero
Spice Girls se destacaram com hits como "If You Wanna Be My Lover" e "Wannabe"
Justin Bieber fez sucesso global durante a adolescência a partir dos anos 2010
Bruno Mars ganhou destaque com uma proposta inovadora, sendo chamado de "Príncipe do Pop"
Bruno Mars é considerado por muitos como o sucessor de Michael Jackson
Taylor Swift se tornou uma das maiores cantoras do pop com reconhecimento mundial
O pop é atualmente o gênero musica

C:\Users\livianavarro-ieg\AppData\Local\Temp\ipykernel_15772\1871694576.py:96: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_juiz = llm_juiz(prompt_formatado)



Avaliação do Juiz (Gemini Pro):
Como juiz de IA, avalio a afirmação:

**SIM**

**Justificativa:**

A "afirmação" apresentada é simplesmente um nome próprio: "Michael Jackson". Um nome próprio, por si só, não é uma proposição que possa ser verdadeira ou falsa no sentido lógico de uma frase completa ou uma afirmação sobre algo.

No entanto, se interpretarmos "correta" no sentido de se o nome se refere a uma entidade real e reconhecida, então a resposta é SIM. Michael Jackson foi uma pessoa real, um artista mundialmente famoso. Portanto, o nome "Michael Jackson" é uma referência correta a essa figura histórica e cultural.

Não há erro factual ou conceitual na apresentação deste nome como uma referência a essa pessoa.

 O Juiz considerou a informação factual.
